<a href="https://colab.research.google.com/github/YNH1985/travel_tide_project/blob/main/travel_tide_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SELECT DISTINCT hotel_name FROM hotels LIMIT 10000;
-- We partner with 2798 Hotels

SELECT DISTINCT trip_airline FROM flights LIMIT 1000;
-- We work together with 355 airlines

SELECT DISTINCT origin_airport FROM flights LIMIT 1000;
SELECT DISTINCT destination_airport FROM flights LIMIT 1000;
-- We bring you from 144 places to 219 destinations!
-- (Why more destinations than origins? No direct return flights?)


-- I want to check if there is a return flight for every origin in the data set:
-- There are more destination than returns. Maybe because this is not the full data set or we have cancelled flights etc.

Descriptive Analysis Part 1

-- Generally speaking only around 6% of our customers are seniors older than 64 years old.
-- For that group we can offer High Quality perks, as the costs for perks would be manageable.

-- The vast majority (588k) of our customers are young adults defined as people between 18 and 44 years old and > 36% of those have children!

-- Therefore, children are a major group of indirect customers! We cannot say how many children exactly our customers have because
-- we do not have data for the number of their kids and there might be duplicates if their mother and there father are both our customers.
-- As an estimate we can use our findings of 0,3671 * 588712 = 211116.
-- (i.e. percentage of young adults with children times group size of young adults).
-- Young adults with children (44 years old, that have at least one kid) add up to 211116.

-- Users Table
SELECT * FROM users LIMIT 1000;

--Birth dates to age
SELECT
    DATE_PART('year', AGE(CURRENT_DATE, birthdate)) AS age
FROM users LIMIT 1000;

-- segmentation of age groups
SELECT
    COUNT(*) FILTER (WHERE DATE_PART('year', AGE(CURRENT_DATE, birthdate)) BETWEEN 0 AND 17) AS underaged,
    COUNT(*) FILTER (WHERE DATE_PART('year', AGE(CURRENT_DATE, birthdate)) BETWEEN 18 AND 44) AS young_adults,
    COUNT(*) FILTER (WHERE DATE_PART('year', AGE(CURRENT_DATE, birthdate)) BETWEEN 45 AND 64) AS older_adults,
    COUNT(*) FILTER (WHERE DATE_PART('year', AGE(CURRENT_DATE, birthdate)) >= 65) AS seniors
FROM users
WHERE birthdate IS NOT NULL;

-- I want to know the percentage of those customer with children (but only for young_adults and adults)
WITH base AS (
    SELECT
        DATE_PART('year', AGE(CURRENT_DATE, birthdate)) AS age,
        has_children
    FROM users
    WHERE birthdate IS NOT NULL
)
SELECT
    age_group,
    ROUND(
        100.0 * COUNT(*) FILTER (WHERE has_children = true)
        / COUNT(*),
        2
    ) AS percent_with_children
FROM (
    SELECT
        CASE
            WHEN age BETWEEN 18 AND 44 THEN 'young_adults'
            WHEN age BETWEEN 45 AND 64 THEN 'older_adults'
        END AS age_group,
        has_children
    FROM base
    WHERE age BETWEEN 18 AND 64
) t
GROUP BY age_group
ORDER BY age_group;



Descriptive Analysis Part 2

-- HOTELS:
-- What are the 10 most popular hotels?
-- Include the information about the average duration of stay and average price before the discount.
-- I also considered negative values for nights as 1.

SELECT
    hotel_name,
    SUM(GREATEST(nights, 1)) AS nights_spent,
    ROUND(AVG(GREATEST(nights, 1)), 2) AS avg_stay_duration,
    ROUND(
        SUM(hotel_per_room_usd * GREATEST(nights, 1))
        / SUM(GREATEST(nights, 1)),
        2
    ) AS avg_price_before_discount
FROM hotels
GROUP BY hotel_name
ORDER BY nights_spent DESC
LIMIT 10;

-- The same for most expensive hotels (top 10)

SELECT
    hotel_name,
    ROUND(
        SUM(hotel_per_room_usd * GREATEST(nights, 1))
        / SUM(GREATEST(nights, 1)),
        2
    ) AS avg_price_per_night,
    SUM(GREATEST(nights, 1)) AS total_nights
FROM hotels
GROUP BY hotel_name
HAVING SUM(GREATEST(nights, 1)) > 0
ORDER BY avg_price_per_night DESC
LIMIT 10;

-- The hotels with the longest stays.
SELECT
    hotel_name,
    ROUND(AVG(GREATEST(nights, 1)), 2) AS avg_stay_duration,
    COUNT(*) AS bookings
FROM hotels
GROUP BY hotel_name
ORDER BY avg_stay_duration DESC
LIMIT 10;


FLIGHTS:
-- What is the most used airline in the last 6 months of recorded data?
-- What is the average number of seats booked on flights via TravelTide?

SELECT trip_airline, COUNT (*) as total_flights_last_6_months,
ROUND(AVG(seats), 2) AS avg_seats_booked
FROM flights
WHERE departure_time >= (SELECT MAX (departure_time) FROM flights) - INTERVAL '6 MONTHS'
GROUP BY trip_airline
ORDER BY total_flights_last_6_months DESC
LIMIT 10
;






WITH origins AS (
    SELECT DISTINCT origin_airport AS airport
    FROM flights
),
destinations AS (
    SELECT DISTINCT destination_airport AS airport
    FROM flights
)

SELECT
    COUNT(DISTINCT o.airport) FILTER (WHERE d.airport IS NOT NULL) AS overlapping,
    COUNT(DISTINCT o.airport) FILTER (WHERE d.airport IS NULL) AS origin_only,
    COUNT(DISTINCT d.airport) FILTER (WHERE o.airport IS NULL) AS destination_only
FROM origins o
FULL OUTER JOIN destinations d
    ON o.airport = d.airport;

-- We have already more than ONE Million customers
SELECT COUNT (DISTINCT user_id) FROM users;




